## Import

In [111]:
import os, random
import numpy as np
import pandas as pd

from ase.io import read

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, Dataset, DataLoader

from tqdm.auto import tqdm

np.set_printoptions(threshold=np.inf)

In [112]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)

seed_everything(42) # Seed 고정

## Pre-Processing

In [113]:
train = read('./train.xyz', format='extxyz', index=':') # 전체 데이터 불러오기
test = read('./test.xyz', format='extxyz', index=':')
sample = pd.read_csv('./sample_submission.csv')

In [114]:
print(f"The number of data: {len(train)}")
train[0]

The number of data: 22510


Atoms(symbols='N24Si24', pbc=True, cell=[8.52238831, 8.52238831, 8.52238831], forces=..., calculator=SinglePointCalculator(...))

In [115]:
sequence_train, symbols, positions_x, positions_y, positions_z, forces, energies = [], [], [], [], [], [], []

for i in range(len(train)):
    mole = train[i] # 각 분자
    
    atoms = len(mole) # 원자 개수
    sequence_train.append(atoms)
    
    position = mole.get_positions() # 원자 위치 정보
    force = mole.get_forces() # label 1
    
    energy = mole.get_total_energy() # label 2
    energies.append(energy)
    
    for j in range(len(mole)): # 각 원자에 대해
        atom = mole[j]
        
        positions_x.append(position[j][0])
        positions_y.append(position[j][1])
        positions_z.append(position[j][2])
        forces.append(force[j])
        
train_df = pd.DataFrame({'position_x': positions_x, 'position_y':positions_y, 'position_z':positions_z, 'force':forces})        
train_df.head()

,position_x,position_y,position_z,force
0,1.591737,4.200483,7.832245,"[-1.9364797, -2.75540073, 0.90898967]"
1,5.640802,2.305094,4.606757,"[1.77046974, -0.17350153, -1.99398617]"
2,6.672786,8.483263,2.981881,"[-2.05488716, -0.29381591, -0.89173793]"
3,1.908548,0.147931,1.741693,"[-0.89207197, -0.8143158, -1.36426899]"
4,4.375650,6.837884,1.948188,"[-4.65938123, -0.77685475, -3.07403915]"


In [116]:
sequence_test, positions_x, positions_y, positions_z = [], [], [], []

for i in range(len(test)):
    mole = test[i] # 각 분자
    
    atoms = len(mole) # 원자 개수
    sequence_test.append(atoms)
    
    position = mole.get_positions() # 원자 위치 정보
    
    for j in range(len(mole)): # 각 원자에 대해
        atom = mole[j]
        
        positions_x.append(position[j][0])
        positions_y.append(position[j][1])
        positions_z.append(position[j][2])
        
test_df = pd.DataFrame({'position_x': positions_x, 'position_y':positions_y, 'position_z':positions_z, 'force':None})        
test_df.head()

,position_x,position_y,position_z,force
0,9.671275,8.734431,6.151755,None
1,1.676806,2.238918,5.270450,None
2,10.358608,4.824889,9.174357,None
3,4.370620,5.391541,9.812298,None
4,2.453404,10.449967,9.906622,None


## [Force] Hyperparameter Setting

In [117]:
# 하이퍼파라미터
input_size = 3  # feature 개수
hidden_size = 512
output_size = 3 # target 개수
num_epochs = 20
batch_size = 128
learning_rate = 0.001

## [Force] Dataset

In [118]:
class ForceDataset(Dataset):
    def __init__(self, df, mode='test'):
        self.df = df
        self.mode = mode
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        
        pos_x = self.df.loc[idx, 'position_x']
        pos_y = self.df.loc[idx, 'position_y']
        pos_z = self.df.loc[idx, 'position_z']
        
        inputs = torch.tensor([pos_x, pos_y, pos_z], dtype=torch.float32)
        
        if not self.mode == 'test':
            label = torch.tensor(self.df.loc[idx, 'force'], dtype=torch.float32)
            return inputs, label
        else:
            return inputs

In [119]:
train_dataset = ForceDataset(train_df, 'train')
test_dataset = ForceDataset(test_df, 'test')

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## [Force] Model

In [123]:
class ForceModel(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(ForceModel, self).__init__()
        
        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.GELU(),
            nn.Dropout(0.2),
            
            nn.Linear(hidden_size, 256),
            nn.BatchNorm1d(256),
            nn.GELU(),
            nn.Dropout(0.2),
            
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.GELU(),
            nn.Dropout(0.2),

            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.GELU(),
            nn.Dropout(0.2),
            
            nn.Linear(64, 3)
        )
    
    def forward(self, x):
        y = self.layers(x)
        
        return y

In [124]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"current device is {device}")

model = ForceModel(input_size, hidden_size).to(device)
criterion = nn.MSELoss()
optimizer = optim.RAdam(model.parameters(), lr=learning_rate)

current device is cuda


## [Force] Train

In [125]:
print("Training Start!")

model.train()
for epoch in range(num_epochs):
    print(f"{epoch+1}/{num_epochs} epoch..")
    for inputs, labels in tqdm(train_loader):
        optimizer.zero_grad()
        
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
print("Training Complete!")        

Training Start!
1/20 epoch..


  0%|          | 0/10039 [00:00<?, ?it/s]

100%|██████████| 10039/10039 [02:27<00:00, 68.27it/s]


2/20 epoch..


100%|██████████| 10039/10039 [02:26<00:00, 68.51it/s]


...

19/20 epoch..


100%|██████████| 10039/10039 [02:11<00:00, 76.31it/s]


20/20 epoch..


100%|██████████| 10039/10039 [02:11<00:00, 76.11it/s]

Training Complete!


## [Force] Inference

In [126]:
print("Inference Start!")

model.eval()

preds = []
with torch.no_grad():
    for inputs in tqdm(test_loader):
        inputs = inputs.to(device)
        outputs = model(inputs)
        
        pred = outputs.detach().cpu().numpy()
        preds.extend(pred)

print("Inference Complete!")
len(preds)

Inference Start!


100%|██████████| 2307/2307 [00:13<00:00, 172.15it/s]

Inference Complete!


295234

## [Force] Submission

In [127]:
test_df['force'] = preds # 예측 결과 저장

In [128]:
# 한 분자가 몇 개의 원자로 이루어져 있는지에 따라 범위를 생성
bundles_train, bundles_test = [], []

flag = 0
for size in sequence_train:
    bundles_train.append((flag, flag+size))
    flag += size

flag = 0
for size in sequence_test:
    bundles_test.append((flag, flag+size))
    flag += size    

In [129]:
preds_force = []

for start, end in bundles_test:
    preds_force.append(np.vstack(preds[start:end])) # 2차원 array로 저장

sample['force'] = preds_force
sample

,ID,energy,force
0,TEST_0000,0,"[[0.016566137, 0.008062972, -0.0032145893], [0..."
1,TEST_0001,0,"[[0.016564596, 0.008066291, -0.0032059941], [0..."
2,TEST_0002,0,"[[0.01656362, 0.008068623, -0.0032033846], [0...."
3,TEST_0003,0,"[[0.016560733, 0.008077284, -0.0031912506], [0..."
4,TEST_0004,0,"[[0.016555855, 0.008091342, -0.0031667652], [0..."
...,...,...,...
4096,TEST_4096,0,"[[-0.0026930198, -0.016181216, -0.0063688383],..."
4097,TEST_4097,0,"[[0.016662741, 0.0074286945, -0.0029525175], [..."
4098,TEST_4098,0,"[[0.016700078, 0.006556238, -0.0031741373], [0..."
4099,TEST_4099,0,"[[0.016700875, 0.0066149356, -0.0031449243], [..."


## [Energy] Preprocessing

In [130]:
# 'force' 컬럼의 값을 분해하여 각각의 행으로 만듦
force_df = train_df['force'].apply(pd.Series)
force_df.columns = [f'force_{i}' for i in range(3)]

# 분해한 'force' 컬럼을 추가
train_df = train_df.drop('force', axis=1).join(force_df)

# 'force' 컬럼의 값을 분해하여 각각의 행으로 만듦
force_df = test_df['force'].apply(pd.Series)
force_df.columns = [f'force_{i}' for i in range(3)]

# 분해한 'force' 컬럼을 추가
test_df = test_df.drop('force', axis=1).join(force_df)
test_df.head()

,position_x,position_y,position_z,force_0,force_1,force_2
0,9.671275,8.734431,6.151755,0.016566,0.008063,-0.003215
1,1.676806,2.238918,5.270450,0.016476,0.006073,-0.004190
2,10.358608,4.824889,9.174357,0.016541,0.008041,-0.003039
3,4.370620,5.391541,9.812298,0.016566,0.008314,-0.003383
4,2.453404,10.449967,9.906622,0.016604,0.008134,-0.003935


In [131]:
# 데이터프레임에서 값 추출
sequences_train = [train_df.iloc[start:end].values for start, end in bundles_train]
sequences_test = [test_df.iloc[start:end].values for start, end in bundles_test]

## [Energy] Hyperparameter Setting

In [132]:
input_size = 6  # feature 개수
hidden_size = 512
output_size = 1 # target 개수
num_epochs = 30
batch_size = 256
learning_rate = 0.001

## [Energy] Dataset

In [133]:
# 패딩을 사용하여 모든 시퀀스의 길이를 동일하게 만듦
max_len = max(seq.shape[0] for seq in sequences_train)
padded_sequences = [np.vstack([seq, np.zeros((max_len - seq.shape[0], 6))]) for seq in sequences_train]

# 패딩된 시퀀스를 2차원 배열로 변환
padded_array_train = np.stack(padded_sequences)
X_tensor_train = torch.tensor(padded_array_train, dtype=torch.float32)
y_tensor_train = torch.tensor(energies, dtype=torch.float32).view(-1, 1)
train_dataset = TensorDataset(X_tensor_train, y_tensor_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 패딩을 사용하여 모든 시퀀스의 길이를 동일하게 만듦
max_len = max(seq.shape[0] for seq in sequences_test)
padded_sequences = [np.vstack([seq, np.zeros((max_len - seq.shape[0], 6))]) for seq in sequences_test]

# 패딩된 시퀀스를 2차원 배열로 변환
padded_array_test = np.stack(padded_sequences)
X_tensor_test = torch.tensor(padded_array_test, dtype=torch.float32)
test_dataset = TensorDataset(X_tensor_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## [Energy] Model

In [134]:
# BiLSTM 모델 정의
class EnergyModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=2, dropout_rate=0.2):
        super(EnergyModel, self).__init__()
        
        # Bidirectional LSTM with Dropout
        self.lstm = nn.LSTM(input_size, 
                            hidden_size, 
                            num_layers, 
                            batch_first=True, 
                            dropout=dropout_rate,
                            bidirectional=True)
        
        # Bidirectional LSTM이므로 hidden_size 조정
        self.linear = nn.Sequential(
            nn.Linear(hidden_size * 2, hidden_size),
            nn.GELU(),
            nn.BatchNorm1d(hidden_size),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size, 1)
        )
        
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        energy = self.linear(lstm_out[:, -1, :])
        return energy

In [135]:
# 모델, 손실 함수, 옵티마이저 초기화
model = EnergyModel(input_size, hidden_size).to(device)
criterion = nn.MSELoss()
optimizer = optim.RAdam(model.parameters(), lr=learning_rate)

## [Energy] Train

In [136]:
print("Training Start!!")

# 학습
model.train()
for epoch in range(num_epochs):
    print(f"{epoch+1}/{num_epochs} epoch..")    
    for inputs, labels in tqdm(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
print("Training Complete!")        

Training Start!!
1/30 epoch..


100%|██████████| 88/88 [00:26<00:00,  3.37it/s]


2/30 epoch..


100%|██████████| 88/88 [00:25<00:00,  3.39it/s]


3/30 epoch..


...

29/30 epoch..


100%|██████████| 88/88 [00:26<00:00,  3.34it/s]


30/30 epoch..


100%|██████████| 88/88 [00:26<00:00,  3.33it/s]

Training Complete!


## [Energy] Inference

In [137]:
print("Inference Start!")

model.eval()

preds = []
with torch.no_grad():
    for inputs in tqdm(test_loader):
        inputs = inputs[0].to(device)

        outputs = model(inputs)
        pred = outputs.detach().cpu().numpy()

        preds.extend(pred)

print("Inference Complete!")        
len(preds)        

Inference Start!


100%|██████████| 17/17 [00:01<00:00,  9.63it/s]

Inference Complete!


4101

## [Energy] Submission

In [138]:
preds = [pred.item() for pred in preds]
sample['energy'] = preds
sample

,ID,energy,force
0,TEST_0000,-639.474976,"[[0.016566137, 0.008062972, -0.0032145893], [0..."
1,TEST_0001,-638.841003,"[[0.016564596, 0.008066291, -0.0032059941], [0..."
2,TEST_0002,-638.773926,"[[0.01656362, 0.008068623, -0.0032033846], [0...."
3,TEST_0003,-639.133118,"[[0.016560733, 0.008077284, -0.0031912506], [0..."
4,TEST_0004,-638.215271,"[[0.016555855, 0.008091342, -0.0031667652], [0..."
...,...,...,...
4096,TEST_4096,-239.591095,"[[-0.0026930198, -0.016181216, -0.0063688383],..."
4097,TEST_4097,-241.008331,"[[0.016662741, 0.0074286945, -0.0029525175], [..."
4098,TEST_4098,-241.683258,"[[0.016700078, 0.006556238, -0.0031741373], [0..."
4099,TEST_4099,-240.737625,"[[0.016700875, 0.0066149356, -0.0031449243], [..."


In [139]:
sample.to_csv('E:/samsung_01/samsumg_01/models/baseline_submission_02.csv', index=False)